# AKARI-SEP Generate hole region files

## I. Produce simple 10 arcsec holes. 
First we produce the simplest possible hole regions. For every star brighter than 16 Mag it puts a 10 arcsec circle. 

We then go on to produce varying size holes for individual pristine catalogues based on parameters computed by Seb Oliver's IDL code.

In [ ]:
from herschelhelp_internal import git_version
print("This notebook was run with herschelhelp_internal version: \n{}".format(git_version()))

In [ ]:
from herschelhelp_internal import starmask
from pymoc import MOC

import time
import os

In [ ]:
OUT_DIR = os.environ.get('OUT_DIR', "./data")
SUFFIX = os.environ.get('SUFFIX', time.strftime("_%Y%m%d"))
SUFFIX = SUFFIX #+ '_WARNING-MADE-WITH-Lockman-SWIRE-PARAMS'

try:
    os.makedirs(OUT_DIR)
except FileExistsError:
    pass

field = 'AKARI-SEP'

First we list all the fields including the ra and dec of the first star in the field. We do this because, due to a peculiarity of the pyregion code, we must supply an image header to produce mocs.

In [ ]:


#Field names plus ra dec of first circle
fields= {
# Field                RA              DEC
'AKARI-NEP':          [274.654402036,  65.7962520276 ],
'AKARI-SEP':          [72.2316923316, -54.380443672  ],
'Bootes':             [216.431700722,  32.401081899  ],
'CDFS-SWIRE':         [51.0227099923, -29.8185285737 ],
'COSMOS':             [149.295925951,  1.08212668291 ],
'EGS':                [217.276981956,  53.6441519854 ],
'ELAIS-N1':           [247.096600963,  55.1757687739 ],
'ELAIS-N2':           [248.424493154,  39.1274077489 ],
'ELAIS-S1':           [7.10625839472, -43.8632559768 ],
'GAMA-09':            [129.076050945, -2.23171513025 ],
'GAMA-12':            [172.84437099,  -0.482115877707],
'GAMA-15':            [211.756497623, -2.28573712848 ],
'HDF-N':              [190.259734752,  62.205265532  ],
'Herschel-Stripe-82': [353.751913281, -7.10891111165 ],
'Lockman-SWIRE':      [161.942787703,  59.0563805825 ],
'NGP':                [192.899559129,  22.0990890388 ],
'SA13':               [197.895801254,  42.4400105492 ],
'SGP':                [334.297748942, -34.5037863499 ],
'SPIRE-NEP':          [266.334305546,  68.7904496043 ],
'SSDF':               [341.577544902, -59.1868365369 ],
'xFLS':               [261.387059958,  58.0184602211 ],
'XMM-13hr':           [203.318355937,  37.4745777866 ],
'XMM-LSS':            [32.9413834032, -6.02293494708 ]}



Then we generate a region file to define all the holes. At present this is very crude and simply puts a 10 arcsec hole over every star brighter than 16 Mag.

In [ ]:
starmask.create_holes('../../dmu0/dmu0_GAIA/data/GAIA_{}.fits'.format(field),
                      'data/10_arcsec_holes_{}.reg'.format(field),
                      '../../dmu2/dmu2_field_coverages/{}_MOC.fits'.format(field))
    


Then we convert the region files to MOC format. We recomend against using the MOC since, even at this high order, it doesn't capture shape that well. This will become increasingly significant if we have smaller or more detailed shapes in the future.

In [ ]:
ORDER=16
starmask.reg2moc('data/10_arcsec_holes_{}.reg'.format(field),
            '../../dmu2/dmu2_field_coverages/{}_MOC.fits'.format(field),
            'data/10_arcsec_holes_{}_O'.format(field) + str(ORDER) + '_MOC.fits',
            ra_typ=fields[field][0],
            dec_typ=fields[field][1],
            order=ORDER)

## II Produce magnitude varying holes
Then we produce a varying hole parameterised by Seb's code. We define an annulus with a 1 arcsec circle at the centre and an outer radius r_50 (the radius at which the artefact density goes over 0.5 x background density) = 10^(A + B x mag).

In [ ]:
# Run with parameters from IDL code. The IDL output files are in ./data/

per_catalogue_params = [
#     Field       Band        A           B             magnitude limit
 
    [field, 'irac_i1',          2.47148,    -0.100727,   16], # Using 18 as faint limit.

]   

#We previously took a moc of the survey to only make holes on that area, which 
#was a mistake as it led to missing holes for the XID+ run


In [ ]:
for pristine_cat in per_catalogue_params:
    starmask.create_holes('../../dmu0/dmu0_GAIA/data/GAIA_' + pristine_cat[0] + '.fits',    #GAIA catalogue
                          'data/holes_' + pristine_cat[0] + '_' + pristine_cat[1] + SUFFIX + '.reg', #output file
                          '../../dmu2/dmu2_field_coverages/' + field + '_MOC.fits',         #moc of region in which to produce holes
                          AB = [pristine_cat[2],pristine_cat[3]],                           #radius as f of mag params
                          mag_lim = pristine_cat[4])                                        #upper magnitude limit

Again we convert the region files to MOC format.

In [ ]:
ORDER=16
for pristine_cat in per_catalogue_params:
    starmask.reg2moc('data/holes_' + field + '_' + pristine_cat[1] + SUFFIX + '.reg',
            '../../dmu2/dmu2_field_coverages/' + field + '_MOC.fits',
            'data/holes_' + field + '_' + pristine_cat[1] + '_O' + str(ORDER) + SUFFIX + '.fits',
            ra_typ=fields[field][0],
            dec_typ=fields[field][1],
            order=ORDER)